Hi there! 👋 

If you’re looking for an easy-to-use guide that can help you automate different stages of software testing, you’ve come to the right place. In this article series, I’ll show you how to use large language models and LangChain to test your software applications. 🤖 This beginner-friendly tutorial is the first part in the series, so be sure to check back for more helpful tips and tricks! 🎉

First I will install the libraries which we we will be using.

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00


In [2]:
!pip install pydantic

In [3]:
!pip install typing_extensions==4.5.0

In [4]:
!pip install -U git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-slisv7yw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-slisv7yw
  Resolved https://github.com/huggingface/transformers.git to commit 66a378429d0e085e4e72bc63a4147889a3b65a14
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7283286 sha256=045a78573b9388bb98cb1f458a98d33d0e9bd958cc25654f938fa9d5d9d2a333
  Stored in directory: /tmp/pip-ephem-wheel-cache-0buv6vxy/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.1
    Uninstalling transformers-4.30.1:
      Successfully uninstalled transformers-4.3

In [5]:
!pip install -U git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-rjzoy0rs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-rjzoy0rs
  Resolved https://github.com/huggingface/accelerate.git to commit e0f5e030098aada5e112708eee3537475dea3a83
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for accelerate: filename=accelerate-0.21.0.dev0-py3-none-any.whl size=241803 sha256=36f1ce95df9c3301212654793d2fa69eed27506d46de692c06ac5ca759b3513c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ffspodex/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [6]:
!pip install --upgrade bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 36.2 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [7]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00


## Import the packages

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu')}
  warn(msg)


## The model
In this tutorial, we'll be working with open source Large Language Models (LLM). While proprietary models can sometimes have better performance, open source options like the one we're using today can still accomplish many tasks effectively. Plus, since they're free to use without any subscriptions, you won't need to worry about extra costs while exploring their capabilities.

For our project, we'll utilize the MPT-7B Chat Model. According to Hugging Face's website, the model is a decoder-style transformer trained on one trillion tokens of both English text and code, thanks to MosaicML's efforts. If you want more insights into this specific model, please follow this [link](https://huggingface.co/mosaicml/mpt-7b-chat).

To use the model we will use Huggingface's transformers library to download the model weights and load the model into an object.

In [9]:
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

model = AutoModelForCausalLM.from_pretrained(
  'mosaicml/mpt-7b-chat',
  torch_dtype=torch.bfloat16,
  device_map="auto",
  trust_remote_code=True,
)

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- attention.py
- norm.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- custom_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- modeling_mpt.py
- attention.py
- custom_embedding.py
- adapt_tokenizer.py
- blocks.py
- hf_prefixlm_converter.py
- meta_init_context.py
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Instantiating an MPTForCausalLM model from /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-chat/c53dee01e05098f81cac11145f9bf45feedc5b2f/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `AutoModelForCausalLM.from_pretrained` function from the transformers library allows you to download and use the model you want from the huggingface hub. This function has some tricks up its sleeve. There is a library known as “accelerate” from huggingface that enables you to load big models on small hardware. The accelerate library is integrated with the transformers library and hence you can ask the `AutoModelForCausalLM.from_pretrained` function to distribute the model weights across your available devices (CPU memory, GPU memory and hard drive). This is done in order to make it possible for really big models to get loaded onto your small hardware.

To do this, you need to provide the following parameters:

* torch_dtype=torch.bfloat16
* device_map=“auto”

Do you see the line `trust_remote_code=True` in the previous section? Currently, the “mpt” models are not integrated with the transformers library. So, they don’t work natively. This line executes some remote code to get it going on our end.

For more information on the "accelerate" library you can follow this [link](https://huggingface.co/docs/accelerate/indexhttps://huggingface.co/docs/accelerate/index).

## Using Huggingface models locally with Langchain
Langchain is a python library that enables you to create applications with large language models. It does all the heavy lifting while providing nice APIs to build powerful applications with LLMs.

The [official docs](https://docs.langchain.com/docs/) provide all the information you need about Langchain. So, do read them through to understand what else you can do with this awesome library. Also, this [video series](https://youtu.be/_v_fgW2SkkQ) by Greg Kamradt is another great resource to learn about Langchain and its different use cases.

We will now use the model object created in the previous section with Langchain. To do this, we will first create a “pipeline” using the pipeline function from the transformers library.

In [10]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=70)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


We need to provide the type of action that we want to use the model for. Here it will be “text-generation”. Then we provide the model object and the tokenizer object which we created in the previous sections. Also, let’s not forget to provide the `max_new_tokens` parameter. This one controls the length of the texts that you want your model to generate. The bigger the number, the longer the generated texts and higher the inference time.

I have kept the number low to have some acceptable inference time but you can play around with the token length as much as your hardware allows.

Next, we need to import the langchain related libraries.

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

We will use the `HuggingFacePipeline` class provided by Langchain. Here we ask Langchain to use the pipeline that we created in the previous section.

In [12]:
local_llm = HuggingFacePipeline(pipeline=pipe)

## Prompting
Now comes the fun part. The beauty of LLMs is that you can just “ask” it to do stuff for you and this can be done by providing it with “prompts”. These are texts that instruct the model to generate “useful” output for you. I won’t delve deep into prompt engineering as it’s not the point of this article. Here, we will explore how we can use prompting to generate test cases for a software application. However, if you want to know more about prompt engineering then this is a very cool [resource](https://www.promptingguide.ai/) that you should definitely look at.

Ok. Back to the point. 

Langchain provides us a nice way to feed our prompts to a language model. This is done using the `PromptTemplate` class.


In [13]:
prompt_template = PromptTemplate(
    input_variables=["product"],
    template="Given a {product} with a single login button. Write test scenarios about the {product}?",
)

Using `PromptTemplate` we provide the input variable which is later interpolated in the template. We shall see in a while how this templating will be useful.

Now, let’s create a chain. A chain is a concept in Langchain that allows us to connect different Langchain components or other chains together to perform a task.

To do this, we will use `LLMChain`.

In [14]:
chain = LLMChain(llm=local_llm, prompt=prompt_template)

In the previous section, we introduced two components: `llm`, which is the model we will use, and `prompt`, which is the prompt template we created. Now we will connect these two components using the `LLMChain`.

It’s time to use our prompt in the model and see what it predicts. Earlier, we created a prompt template that we will now use to generate our input. The template was “Given a {product} with a single login button. Write test scenarios about the {product}?” and the stand-in for our input variable was`{product}`. So, when we give a product name like “web application”, Langchain formats our template and inserts this product name into the prompt and then feeds it to the model. It also runs inference and provides us with a formatted output. All of this can be done using the one liner below.

In [15]:
%%time
print(chain.run("web application"))



1. User logs in successfully
2. User enters incorrect login credentials and receives an error message
3. User enters login credentials for a non-existent account and receives an error message
4. User enters login credentials for an account that has been deactivated or disabled
5. User enters login credentials for an account that has been locked
CPU times: user 29.2 s, sys: 444 ms, total: 29.7 s
Wall time: 31.4 s


There we go. We have successfully generated few test cases for a web application. It's cool to note that we didn't provide any elaborate context to the model about the web application but it still came up with some commonly tested scenarios.

## Lending a helping hand to the model
The model sure did a good job in generating some test cases but what if we wanted the test cases to be generated in a certain style/format or what if the generated test cases are not "the way" we want them to be. We can help out the model by providing a few examples of use cases and the related test cases. This should steer the model in the right direction.

Langchain provides another API known as `FewShotPromptTemplate`. By using this API we can pass a prompt with a few examples to the model.

To start with this, let's import the module.

In [16]:
from langchain.prompts.few_shot import FewShotPromptTemplate

Next, we will create a list of example texts as below.

In [17]:
examples = [
    {"Usecase": "Given a web application with a single login button. Write test scenarios about the login button in the web application?",
    "Test cases": 
"""
1. User logs in successfully.
2. User enters incorrect login credentials and receives an error message.
3. User enters login credentials for a non-existent account and receives an error message.
4. User enters login credentials for an account that has been deactivated or disabled.
5. User enters login credentials for an account that has been locked.
"""},
    {"Usecase": "Given a REST API which accepts a POST request. Write test scenarios about the API?",
    "Test cases": 
"""
1. User sends valid request to the API and recieves valid response.
2. User sends an empty body to the API and recieves an error message.
3. User sends a request with invalid fields to the API and recieves an error mesage.
"""},
    {"Usecase": "Given a ETL pipeline which extracts, transforms and loads data from a source table to a target table. Write test scenarios about the data ETL pipeline?",
    "Test cases": 
"""
1. Verify that the data in the target database is the same as the data in the source.
2. Verify that the constraints in the target database is as per the functional specification.
3. Verify that the target database does not has any empty field.
4. Verify that the target database does not has any duplicate values.
"""}
           ]

The examples list contains dictionaries which are our examples. Each dictionary has two fields: “Usecase” and “test cases”. The “Usecase” field has a context or scenario, and the “Test cases” field has sample test cases that correspond to the use case.

Next, we need convert this examples into an object that can be used by Langchain. To do this we will wrap up our input variables and the example template by using the `PromptTemplate` API.

In [18]:
example_prompt = PromptTemplate(input_variables=["Usecase", "Test cases"], template="Usecase: {Usecase}\n{Test cases}")

print(example_prompt.format(**examples[0]))


Usecase: Given a web application with a single login button. Write test scenarios about the login button in the web application?

1. User logs in successfully.
2. User enters incorrect login credentials and receives an error message.
3. User enters login credentials for a non-existent account and receives an error message.
4. User enters login credentials for an account that has been deactivated or disabled.
5. User enters login credentials for an account that has been locked.



Here, we have two input variables : "Usecase" and "Test cases". These are the two fields which have provided in the `examples` list. Our template is "Usecase/Test cases" format.

Then we wrap everything in the `FewShotPromptTemplate` class.

In [19]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Usecase: {input}", 
    input_variables=["input"]
)

The most important things in the code above are the `examples` and the `example_prompt` parameters. These are where we provide the examples list and the example prompt object that we created in the previous section.



This is how the formatted prompt will look like.

In [20]:
print(prompt.format(input="Given a target database which has incoming data from a source csv file.\
Write test cases to verify the data in the target database."))

Usecase: Given a web application with a single login button. Write test scenarios about the login button in the web application?

1. User logs in successfully.
2. User enters incorrect login credentials and receives an error message.
3. User enters login credentials for a non-existent account and receives an error message.
4. User enters login credentials for an account that has been deactivated or disabled.
5. User enters login credentials for an account that has been locked.


Usecase: Given a REST API which accepts a POST request. Write test scenarios about the API?

1. User sends valid request to the API and recieves valid response.
2. User sends an empty body to the API and recieves an error message.
3. User sends a request with invalid fields to the API and recieves an error mesage.


Usecase: Given a ETL pipeline which extracts, transforms and loads data from a source table to a target table. Write test scenarios about the data ETL pipeline?

1. Verify that the data in the targe

We will proceed with the same steps as before i.e. create the `LLMChain` object and run the inference.

In [21]:
chain = LLMChain(llm=local_llm, prompt=prompt)

In [22]:
%%time
print(chain.run("Given a target database which has incoming data from a source csv file.\
                Write test cases to verify the data in the target database."))



1. Verify that the data in the target database is the same as the data in the source csv file.
2. Verify that the data in the target database is in the correct format.
3. Verify that the data in the target database is in the correct structure.
4. Verify that the data in
CPU times: user 2min 26s, sys: 0 ns, total: 2min 26s
Wall time: 2min 26s


Look how the LLM has generated the test cases in a flavour that is similar to what we suggested with the examples in the prompt.

What if we wanted to have something more customized? We can do that too with cleverly engineered prompts. We will try to customize our example test cases in Gherkin format.

Gherkin can be considered as a domain specific language. It contains a set of specific keywords using which we can write business specific use cases. For example let us suppose that there is a web application that we want to test and the use case under test is that the web page should be as per the spec sheet. To write this is Gherkin we we could write it in the following way-

"""
Feature : website layout

    Scenario: verify the website layout
        Given a website page
        When user visits the page
        Then the page should be as per the spec sheet
"""

You see! How we are able to write the test case in simple natural language? This is the beauty of the Gherkin language.

There is also a reason why I chose the Gherkin language for instructing the LLM to generate test cases. Gherkin has a natural language format that helps describe business behavior without going into implementation details. In our present scenario, i.e., test case generation, Gherkin is an excellent way to feed technical business scenarios to the language model. Now that we understand what Gherkin is, let’s build our examples in Gherkin format.

Now that we understand what Gherkin is, let's build our examples in Gherkin format.

In [23]:
examples = [
    {"Usecase": "Given a web application with a single login button. Write test scenarios about the login button in the web application?",
    "Test case": 
"""
Feature: Login button

  Scenario: test the login button
     Given a login button
      When user clicks on the button
      Then login should be successful

  Scenario: test the login button
     Given a login button
      When user enters invalid credentials and clicks on the login button
      Then login should not be successful
"""},
    {"Usecase": "Given a REST API which accepts a POST request. Write test scenarios about the API?",
    "Test case": 
"""
Feature: API endpoint

  Scenario: test the API endpoint with valid request
     Given an API endpoint
      When user sends a valid request
      Then a correct response should be recieved

  Scenario: test the API endpoint with empty body
     Given an API endpoint
      When user sends a request with empty body
      Then an error code should be recieved
"""}
           ]

From here we will follow the usual steps of creating the `PromptTemplate`, `FewShotPromptTemplate` and `LLMChain` objects like before.

In [24]:
example_prompt = PromptTemplate(input_variables=["Usecase", "Test case"], template="Usecase: {Usecase}\n{Test case}")

print(example_prompt.format(**examples[0]))

Usecase: Given a web application with a single login button. Write test scenarios about the login button in the web application?

Feature: Login button

  Scenario: test the login button
     Given a login button
      When user clicks on the button
      Then login should be successful

  Scenario: test the login button
     Given a login button
      When user enters invalid credentials and clicks on the login button
      Then login should not be successful



In [25]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Usecase: {input}", 
    input_variables=["input"]
)

In [26]:
chain = LLMChain(llm=local_llm, prompt=prompt)

then we will ask Langchain to start the inferencing.

In [27]:
%%time
print(chain.run("Given a target database which has incoming data from a source csv file.\
                Write test cases to verify the data in the target database."))



Feature: Data in target database

  Scenario: test data in target database
     Given a target database
      And a source csv file
      When the source csv file is imported into the target database
      Then the data should be present in the target database

  Scenario: test data in target database
     Given a
CPU times: user 1min 50s, sys: 0 ns, total: 1min 50s
Wall time: 1min 50s


Look! how with the help of some clever prompts we steered the LLM to generate test cases in the Gherkin format. 

## Conclusion
I hope this beginner-friendly tutorial has helped you understand how to use large language models and langchain for generating test cases for a software application. By following the framework provided in this article, you can automate the test case desgining stage and make your testing process more efficient. This article is the first part in the series, so stay tuned for more tutorials where we will experiment with integrating LLMs in other stages of software testing.

Please upvote this notebook so that I know that it was helpul to you and it will also make it easier for others to discover this as well.